In [3]:
import access_classes as ac
import comment_thread as ct
from dateutil import parser
import webbrowser
import numpy as np
import networkx as nx
import pandas as pd
from datetime import datetime, timedelta
import yaml

In [2]:
def process_project_dates(project):
    OUTPUT = {}
    threads = ct.main(project , merge=False)
    for thread in threads.values():
        dates = pd.Series(nx.get_node_attributes(thread.graph, 'com_timestamp')).sort_values()
        df = pd.DataFrame({'stamps': dates ,'deltas': (dates - dates[0])})
        df['url'] = df.index.map(lambda com_id: "#".join([thread.data.url, com_id]))
        thresh = df['deltas'].mean() + (2 * df['deltas'].std())
        outliers = df[df['deltas'] > thresh]
        print(thread.post_title)
        print("below thresh:\n", df[df['deltas'] <= thresh][-10:][['stamps', 'deltas']])
        print("extreme outliers:\n", outliers[['stamps', 'deltas']])
        try:
            open_this = outliers.ix[-1, 'url']
        except:
            open_this = df.ix[-1, 'url']
        finally:
            webbrowser.open(open_this, new=2, autoraise=True)
        set_as_last = input("Set as last: ")
        if set_as_last:
            try:
                set_as_last = parser.parse(set_as_last)
                print("Date parsed as: ", set_as_last)
            except:
                print("Parsing failed")
            try:
                OUTPUT[thread.data.url] = set_as_last.date() + timedelta(days=1)
            except:
                OUTPUT[thread.data.url] = set_as_last
    try:
        with open("find_lasts.yaml", 'a') as outfile:
            yaml.dump(OUTPUT, outfile)
    except:
        print("Could not save data")
    finally:
        return OUTPUT

In [4]:
with open("lasts_by_date.yaml", 'r') as infile:
    as_dict = yaml.safe_load(infile)


In [5]:
as_dict

{'http://gilkalai.wordpress.com/2009/03/25/an-open-discussion-and-polls-around-roths-theorem/': datetime.date(2009, 4, 13),
 'http://gilkalai.wordpress.com/2009/07/17/the-polynomial-hirsch-conjecture-a-proposal-for-polymath3/': datetime.date(2009, 8, 3),
 'http://gilkalai.wordpress.com/2009/07/28/polymath3-abstract-polynomial-hirsch-conjecture-aphc/': datetime.date(2009, 8, 1),
 'http://gilkalai.wordpress.com/2009/07/30/the-polynomial-hirsch-conjecture-how-to-improve-the-upper-bounds/': datetime.date(2009, 8, 6),
 'http://gilkalai.wordpress.com/2009/08/09/the-polynomial-hirsch-conjecture-discussion-thread/': datetime.date(2009, 9, 30),
 'http://gilkalai.wordpress.com/2009/10/06/the-polynomial-hirsch-conjecture-discussion-thread-continued/': datetime.date(2009, 11, 7),
 'http://gilkalai.wordpress.com/2009/12/08/plans-for-polymath3/': datetime.date(2010, 1, 7),
 'http://gilkalai.wordpress.com/2010/06/19/the-polynomial-hirsch-conjecture-the-crux-of-the-matter/': datetime.date(2010, 6, 21)

## Old functions

In [ ]:
def process_finals():
    with open("new_lasts.yaml", 'r') as infile:
        OUTPUT = yaml.safe_load(infile.read())
    for url in LASTS[0].keys():
        if url in OUTPUT:
            pass
        else:
            webbrowser.open(url, new=2, autoraise=True)
            last_date = input("Last date or skip: ")
            if last_date:
                try:
                    last_date = parser.parse(last_date)
                    print("Date parsed as: ", last_date)
                except:
                    print("Parsing failed")
                    last_date = None
                finally:
                    OUTPUT[url] = last_date
            else:
                break
    with open("new_lasts.yaml", 'w') as outfile:
        yaml.dump(OUTPUT, outfile)